In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pandas as pd
from scipy import optimize
from scipy import integrate
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)

In [2]:
def store_flat_table_JH_data():

    data_path_1 = r"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/" \
                  r"master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
    pd_raw = pd.read_csv(data_path_1)
    time_idx = pd_raw.columns[4:]
    df_plot = pd.DataFrame({"date": time_idx})
    Country_list = ["Germany", "India", "Italy"]
    for each in Country_list:
        df_plot[each] = np.array(pd_raw[pd_raw["Country/Region"] == each].iloc[:, 4::].sum(axis=0))

    time_idx = [datetime.strptime(each, "%m/%d/%y") for each in df_plot.date]
    time_str = [each.strftime('%Y-%m-%d') for each in time_idx]
    df_plot["date"] = time_idx
    df_plot.to_csv('../data/processed/COVID_small_flat_table.csv', sep=';', index=False)
    print("Data Acquired and Stored as COVID_small_flat_table.csv")


store_flat_table_JH_data()

Data Acquired and Stored as COVID_small_flat_table.csv


In [3]:
def SIR_model_t(SIR,t,beta,gamma):
    S,I,R = SIR #Susceptible, Infectious, or Recovered
    dS_dt = -beta*S*I/N0
    dI_dt = beta*S*I/N0-gamma*I
    dR_dt = gamma*I
    return dS_dt,dI_dt,dR_dt


def slice_data(data,con):
    ydata = np.array(data[con])
    t = np.arange(len(ydata))
    return ydata, t

def fit_odeint(x, beta, gamma):

    '''
    helper function for the integration
    '''
    I0 = ydata[0]
    S0 =N0 - I0
    R0=0
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI

In [4]:
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';')
df_analyse=df_analyse.sort_values('date',ascending=True)
df_analyse=df_analyse[30:130:]
df_analyse= df_analyse.reset_index().drop(["index"], axis=1)
N0 =10000000
popt=[0.3,0.1]
country_list= df_analyse.columns[1:]
for each in country_list:
    ydata, t = slice_data(df_analyse,each)
    fit_odeint(t, *popt)
    popt2, pcov = optimize.curve_fit(fit_odeint, t, ydata, maxfev=1000)
    perr = np.sqrt(np.diag(pcov))
    print('standard deviation errors : ', str(perr), ' start infect:', ydata[0])
    print("Optimal parameters: beta =", popt2[0], " and gamma = ", popt2[1])
    fitted=fit_odeint(t, *popt)
    df_analyse[each+"fit"]= pd.DataFrame(fitted)

df_analyse= df_analyse.reset_index()
df_analyse.head()

C:\Users\axayp\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-3-a697aa4ab543>:3: RuntimeWarning: overflow encountered in double_scalars
  dS_dt = -beta*S*I/N0
<ipython-input-3-a697aa4ab543>:4: RuntimeWarning: overflow encountered in double_scalars
  dI_dt = beta*S*I/N0-gamma*I
C:\Users\axayp\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


standard deviation errors :  [0.01625824 0.0162313 ]  start infect: 16
Optimal parameters: beta = 0.6727889304673567  and gamma =  0.5347011835898832
standard deviation errors :  [0.01020551 0.00984788]  start infect: 3
Optimal parameters: beta = 0.7379126820613023  and gamma =  0.6106646940197323
standard deviation errors :  [0.01526762 0.01523717]  start infect: 20
Optimal parameters: beta = 0.599319424127453  and gamma =  0.4601547804460787


,index,date,Germany,India,Italy,Germanyfit,Indiafit,Italyfit
0,0,2020-02-21,16,3,20,16.000000,3.000000,20.000000
1,1,2020-02-22,16,3,62,19.542433,3.664208,24.428038
2,2,2020-02-23,16,3,155,23.869164,4.475473,29.836446
3,3,2020-02-24,16,3,229,29.153836,5.466354,36.442274
4,4,2020-02-25,16,3,322,35.608532,6.676619,44.510627


In [5]:
import random
color_list = []
for i in range(200):
    var = '#%02x%02x%02x'%(random.randint(0,255),random.randint(0,255),random.randint(0,255))
    color_list.append(var)

In [ ]:
import pandas as pd
import numpy as np

import dash
dash.__version__
from dash import dcc
from dash import html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
df_input_large=df_analyse


fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    Deliver_4: Dynamic dashboard SIR model

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for SIR visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_analyse.columns[2:5]],
        value=['India'], # which are pre-selected
        multi=True
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])


def update_figure(country_list):


    v=0
    my_yaxis={'type':"log",
              'title':'New Population Infected'
          }


    traces = []
    for each in country_list:

        traces.append(dict(x=df_analyse.index,
                                y=df_analyse[each],
                                mode='markers', markers = dict(color= color_list[v]),
                                opacity=0.9,
                                name=each
                        )
                )
        traces.append(dict(x=df_analyse.index,
                        y=df_analyse[each+"fit"],
                        mode='lines',line = dict(color=color_list[v]), opacity=1.0,name=each+'_Fit'))
        v= v+1

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
